In [3]:
# โหลดไฟล์ CSV ใหม่หลังจากรีเซ็ต state
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# กำหนดพาธไฟล์ใหม่
law_path = "./law.csv"
core_service_path = "./core_service.csv"
task_path = "./task.csv"
link_path = "./link.csv"

# โหลดข้อมูลจาก CSV
law_df = pd.read_csv(law_path)
core_service_df = pd.read_csv(core_service_path)
task_df = pd.read_csv(task_path)
link_df = pd.read_csv(link_path)


In [4]:

# กำหนดระดับของแต่ละประเภท
category_levels = {
    "law": 1,
    "core_service": 2,
    "task": 3
}

# กำหนดหมวดหมู่ของแต่ละตาราง
law_df["category"] = "law"
core_service_df["category"] = "core_service"
task_df["category"] = "task"

# รวมข้อมูลและกำหนด level
all_nodes = pd.concat([law_df, core_service_df, task_df], ignore_index=True)
all_nodes["level"] = all_nodes["category"].map(category_levels)

# สร้าง dictionary mapping key -> label และ key -> level
key_to_label = all_nodes.set_index("key")["label"].to_dict()
key_to_level = all_nodes.set_index("key")["level"].to_dict()

# แทนค่า key ด้วย label และ level ในตาราง link
link_df["source_label"] = link_df["source"].map(key_to_label)
link_df["target_label"] = link_df["target"].map(key_to_label)
link_df["source_level"] = link_df["source"].map(key_to_level)
link_df["target_level"] = link_df["target"].map(key_to_level)

# ลบข้อมูลที่ไม่มี label ตรงกับ key ที่ระบุ
link_df = link_df.dropna(subset=["source_label", "target_label"])


In [6]:

# สร้าง Network Graph
G = nx.DiGraph()

# เพิ่มโหนด
for _, row in all_nodes.iterrows():
    G.add_node(row["label"], level=row["level"], category=row["category"])

# เพิ่มการเชื่อมโยง
for _, row in link_df.iterrows():
    G.add_edge(row["source_label"], row["target_label"])

# กำหนดตำแหน่งของโหนดตามระดับ
pos = nx.multipartite_layout(G, subset_key="level")

# แยกข้อมูลสำหรับ Plotly
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

# กำหนดสีของโหนดตามประเภท
node_colors = {"law": "blue", "core_service": "green", "task": "red"}
node_x, node_y, node_text, node_color_list = [], [], [], []

for node, (x, y) in pos.items():
    node_x.append(x)
    node_y.append(y)
    node_text.append(node)
    node_category = next((data["category"] for n, data in G.nodes(data=True) if n == node), "gray")
    node_color_list.append(node_colors.get(node_category, "gray"))

# สร้าง Network Graph ด้วย Plotly
fig = go.Figure()

# เพิ่มเส้นเชื่อมโยง (Edges)
fig.add_trace(go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=1, color="gray"),
    hoverinfo="none",
    mode="lines"
))

# เพิ่มโหนด (Nodes)
fig.add_trace(go.Scatter(
    x=node_x, y=node_y,
    mode="markers+text",
    marker=dict(size=10, color=node_color_list),
    text=node_text,
    textposition="top center",
    hoverinfo="text"
))

# ปรับแต่งรูปแบบของกราฟ
fig.update_layout(
    title="Network Graph (Law -> Core Service -> Task)",
    showlegend=False,
    hovermode="closest",
    margin=dict(b=0, l=0, r=0, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
)

# บันทึกเป็นไฟล์ HTML
html_file_path = "./network_graph.html"
fig.write_html(html_file_path)

# ส่งไฟล์ให้ผู้ใช้ดาวน์โหลด
html_file_path

'./network_graph.html'